# Plan and Execute SRE Agent

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

gpt5 = ChatOpenAI(model="gpt-5")
gpt5mini = ChatOpenAI(model="gpt-5-mini")
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLEAI_API_KEY") # Google AI Studio free api key
)

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "kubernetes" : {
            "command": "npx",
            "args": ["mcp-server-kubernetes"],
            "transport": "stdio",
            "env": {
                "ALLOW_ONLY_NON_DESTRUCTIVE_TOOLS": "true"
            }
        }
    }
)

# NB: The cluster has to be running otherwise the tools won't be available
mcp_tools = await client.get_tools()

In [ ]:
# Tools allowed
tools_allowed = ["kubectl_get", "kubectl_describe", "kubectl_logs", "explain_resource", "list_api_resources", "ping"]

tools = []
for tool in mcp_tools:
    if tool.name in tools_allowed:
        tools.append(tool)

for tool in tools:
    print(f"🔧 {tool.name}: {tool.description}")

## Pattern used

https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/plan-and-execute/

### Agent executor

The agent that executes the commands given by the planner

In [ ]:
from langgraph.prebuilt import create_react_agent

prompt = "You are a helpful assistant."
agent_executor = create_react_agent(gpt5mini, tools, prompt=prompt)

### Planner

In [ ]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    insights: Annotated[List[str], operator.add]
    response: str

In [ ]:
from pydantic import BaseModel, Field

class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

from pydantic import BaseModel, Field

In [ ]:
planner_prompt = """
    You are a planning agent. Your goal is to detect anomalies in a deployed service (kubernetes cluster) by decomposing the task into clear, concrete steps.

    The service you are working with today is described below:
    {app_summary}

    Each step must be self-contained and executable by an external executor or by reasoning over previously gathered results.

    Guidelines:
    - Be concise and avoid unnecessary steps.
    - Each step should contain enough detail so it can be executed independently.
    - The final step must be an "analysis" step that synthesizes the anomaly diagnosis.
"""

app_summary = """
    The application implements a hotel reservation service, build with Go and gRPC, and starting from the open-source project https://github.com/harlow/go-micro-services. The initial project is extended in several ways, including adding back-end in-memory and persistent databases, adding a recommender system for obtaining hotel recommendations, and adding the functionality to place a hotel reservation. 
"""

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

planner_llm_configured = gpt5mini.with_structured_output(Plan)

async def planner():
    return planner_llm_configured.invoke([planner_prompt.format(app_summary=app_summary)])

### Re-Planner